## JOINS

Topics covered in this notebook:

- `AS`
- `INNTER JOINS`
- `OUTER JOINS`
- `FULL JOINS`
- `UNIONS`



### Libraries and function setup to perform queries

In [1]:
# Libraries
import pandas as pd
import sqlite3

cnx = sqlite3.connect('./data/jobs.db')

# Definimos la función para hacer queries.
def sql_query(query):
    return pd.read_sql(query, cnx)

### `AS`

Creates an alias for a column or result.

Note: aliases are only created at the end

In [4]:
query = """
SELECT Name, SUM(Salary) AS total_salary
FROM jobs
GROUP BY Name
"""
sql_query(query)

,Name,total_salary
0,Aaron,2388324
1,Abigail,523376
2,Adam,2300153
3,Adrian,820912
4,Adriana,169895
...,...,...
658,Yolanda,411244
659,Yvette,165454
660,Yvonne,373081
661,Zachary,2080021


One typicall error at the begining is to perform alias at end in HAVING.

Alias is assigned at the very end. Therefore, the above code should not work and it is incorrect.

In [18]:
query = """
SELECT Name, SUM(Salary) AS total_salary
FROM jobs
GROUP BY Name
HAVING total_salary < 40000
"""
sql_query(query)

,Name,total_salary
0,Alvin,36155
1,Belinda,38383
2,Gilbert,30244
3,Jake,30059
4,Kirsten,34761
5,Leon,35273
6,Nina,33505
7,Patty,37318


The correct way would be as follows.

In [20]:
query = """
SELECT Name, SUM(Salary) AS total_salary
FROM jobs
GROUP BY Name
HAVING SUM(Salary) < 40000
"""
sql_query(query)

,Name,total_salary
0,Alvin,36155
1,Belinda,38383
2,Gilbert,30244
3,Jake,30059
4,Kirsten,34761
5,Leon,35273
6,Nina,33505
7,Patty,37318


### `JOINS`

Allow us to combine information from multiple tables together.

The main reason for the different JOIN types is to decide how to deal with information only present in one of the joined tables.

### Until now we were playing with a database with only one table, but in the real world we will have many in same database.

Let´s view all the tables present

In [39]:
cnx_logins_registration = sqlite3.connect('./data/logins_registrations.db')
crsr = cnx_logins_registration.cursor()

# Definimos la función para hacer queries.
def sql_query_joins(query):
    return pd.read_sql(query, cnx_logins_registration)

In [40]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

registrations
logins


**Registrations Table**

In [47]:
query = """
SELECT *
FROM registrations

"""
sql_query_joins(query)

,reg_id,Name
0,1,Pepe
1,2,Luis
2,3,Amelia
3,4,Susana


**Logins Table**

In [48]:
query = """
SELECT *
FROM logins

"""
sql_query_joins(query)

,log_id,Name
0,1,Laura
1,2,Pepe
2,3,Fernando
3,4,Luis


#### `INNER JOIN`


<div>
    <img src="https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/INNER_JOIN" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>

In [43]:
query = """

SELECT *
FROM registrations AS r
INNER JOIN logins AS l
ON r.Name = l.Name

"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,1,Pepe,2,Pepe
1,2,Luis,4,Luis


#### `FULL JOIN`




<div>
    <img src="https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/FULL_JOIN" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>

In [45]:
query = """

SELECT *
FROM registrations AS r
FULL OUTER JOIN logins AS l
ON r.Name = l.Name

"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,1.0,Pepe,2.0,Pepe
1,2.0,Luis,4.0,Luis
2,3.0,Amelia,NaN,None
3,4.0,Susana,NaN,None
4,NaN,None,1.0,Laura
5,NaN,None,3.0,Fernando


### `OUTER EXCLUDING JOIN` | FULL OUTER JOIN with WHERE  

<div>
    <img src="https://www.codeproject.com/KB/database/Visual_SQL_Joins/OUTER_EXCLUDING_JOIN.png" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>

In [46]:
# Get rows unique to either table (rows not found in both tables)
query = """

SELECT *
FROM registrations AS r
FULL OUTER JOIN logins AS l
ON r.Name = l.Name
WHERE r.reg_id IS null OR 
l.log_id IS null

"""
sql_query_joins(query)


,reg_id,Name,log_id,Name
0,3.0,Amelia,NaN,None
1,4.0,Susana,NaN,None
2,NaN,None,1.0,Laura
3,NaN,None,3.0,Fernando


#### `LEFT JOIN`



<div>
    <img src="https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/LEFT_JOIN" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>


In [49]:
query = """

SELECT *
FROM registrations AS r
LEFT OUTER JOIN logins AS l
ON r.Name = l.Name


"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,1,Pepe,2.0,Pepe
1,2,Luis,4.0,Luis
2,3,Amelia,NaN,None
3,4,Susana,NaN,None


#### `LEFT EXCLUDING JOIN`


<div>
    <img src="https://i.stack.imgur.com/oEEDZ.png" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>



In [51]:
# Gets unique rows to Table A

query = """

SELECT *
FROM registrations AS r
LEFT OUTER JOIN logins AS l
ON r.Name = l.Name
WHERE l.Name IS null


"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,3,Amelia,None,None
1,4,Susana,None,None


#### `RIGHT JOIN`




<div>
    <img src="https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/RIGHT_JOIN" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>


In [52]:
query = """

SELECT *
FROM registrations AS r
RIGHT OUTER JOIN logins AS l
ON r.Name = l.Name


"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,1.0,Pepe,2,Pepe
1,2.0,Luis,4,Luis
2,NaN,None,1,Laura
3,NaN,None,3,Fernando


#### `RIGHT EXCLUDING JOIN`

<div>
    <img src="https://i.stack.imgur.com/rnv23.png" alt="Image 1">
    <img src="../assets/reg_log_image.png" alt="Image 2" style="width: 400px; display: inline-block;">
</div>


In [53]:
query = """

SELECT *
FROM registrations AS r
RIGHT OUTER JOIN logins AS l
ON r.Name = l.Name
WHERE r.Name is null

"""
sql_query_joins(query)

,reg_id,Name,log_id,Name
0,None,None,1,Laura
1,None,None,3,Fernando


### `UNION`

In [54]:
# Used to combine the result-set of two or more SELECT statements.
# It basically serves to directly concatenate two results together, essentially "pasing" them together

query = """

SELECT Name
FROM registrations
UNION
SELECT Name
FROM
logins


"""
sql_query_joins(query)

,Name
0,Amelia
1,Fernando
2,Laura
3,Luis
4,Pepe
5,Susana


### Udemy challenge where film_actor had id of films and actor and acted as glue to film table and actor table

In [ ]:

""" 
SELECT title,first_name,last_name
FROM film_actor
JOIN film ON film_actor.film_id = film.film_id
JOIN actor ON film_actor.actor_id = actor.actor_id
WHERE first_name ='Nick' AND last_name ='Wahlberg'
"""